In [1]:
import os
import pickle

import mlflow
from mlflow import log_metric, log_param, log_artifacts

import numpy as np
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

In [2]:
server = 'http://localhost:5000'
experiment = 'linear'
mlflow.set_tracking_uri(server)
mlflow.set_experiment(experiment)

In [3]:
grid = ParameterGrid({'C': np.e**np.linspace(-3, 3, 10), 
                      'penalty': ['l1', 'l2']})

for item in grid:
    mlflow.start_run()

    X, Y = make_classification(n_samples=400, n_features=2, 
                               n_informative=2, n_classes=2, 
                               n_redundant=0,
                               n_clusters_per_class=1,
                               random_state=0)
    X_train, X_test, Y_train, Y_test = train_test_split(
        X, Y, test_size=100, random_state=0)

    log_param("C", item['C'])
    log_param("penalty", item['penalty'])

    model = LogisticRegression(C=item['C'], 
                               penalty=item['penalty'], solver='saga')

    model.fit(X_train, Y_train)

    log_metric("train acc", model.score(X_train, Y_train))
    log_metric("test acc", model.score(X_test, Y_test))

    mlflow.end_run()